In [1]:
import pandas as pd
import requests
import json
from datetime import datetime, timezone
import numpy as np

In [2]:
with open('..\\..\\CypressAutomation\\cypress\\fixtures\\Zephyr.json') as f:
    requestData = json.load(f)
requestData['slots'] = "B"
requestData['format_output'] = "json"
requestData['target'] = "api"
print(requestData)

{'email': 'AstonUniversity', 'password': 'Xo08R83d43e0Kk6', 'dateStart': '20210917000', 'dateEnd': '202109200000', 'sensors': ['814', '821'], 'slots': 'B', 'format_output': 'json', 'target': 'api'}


In [3]:
zephyr_jsons = []

for zephyr_id in requestData['sensors']:
    url = "https://data.earthsense.co.uk/dataForViewBySlots/%s/%s/%s/%s/%s/%s/def/%s/%s" % (requestData['email'], requestData['password'], zephyr_id,requestData['dateStart'],requestData['dateEnd'],requestData['slots'],requestData['format_output'],requestData['target'])
    response = requests.get(url)
    if response.status_code == 200:
        zephyr_jsons.append(response.json()['slotB'])
    else:
        print("HTTP request error code: " + response.status_code)

# json to dt

In [4]:
#extracting and preparing the dataframe from the json objects
dfList = []
for jsonObject in zephyr_jsons:
    df_temp = pd.DataFrame.from_records(jsonObject)
    df_temp.drop('header', axis =0, inplace=True)
    df_temp.drop('data_hash', axis =0, inplace=True)
    df_temp.drop('UTS', axis=1, inplace=True)

    #explode function transform each element of a list to a row.
    #we can apply it to all the columns assuming they have the same number of elements in each list 
    df_temp = df_temp.apply(pd.Series.explode)
    
    dfList.append(df_temp)

measurement_dictionary = {k:v for k,v in zip(requestData['sensors'],dfList)}
dfList = [] # clear dataframe list


# Splitting dataframe into days

In [5]:
def dataSplit(df_temp,dateString):
    
    data = {} #intialise empty dictionary to store each day of locations
    
    df_temp = measurement_dictionary[key]

    # using the dates which are already supplied. This strategy in the line below converts them and rounds down to date using 'd' flag
    # This strategy (line below) will keep just the date
    df_temp['day'] = pd.to_datetime(df_temp[dateString], dayfirst=True, errors='coerce').dt.date

    the_unique_dates = df_temp['day'].unique()
    # this gives the same result as the for loop below
    
    #splitting the dataframe into separate days
    #for each day in unique dates set:
    for day in the_unique_dates:
        try:
            # In my code below I assign the subset of records to a new dataframe called dft
            # create 'midnight' timestamps
            timestampKey = int((pd.to_datetime(day, errors='coerce')).timestamp())

            # select the records for this day
            dft = df_temp[df_temp['day']==day]

            # #drop the date column to save space (we don't need this anymore)
            #dft = dft.drop(dateString, axis=1)
            dft = dft.drop("day", axis=1)
            dft = dft.dropna() #drop null values
            
            data[timestampKey] = dft

        except KeyError as e:
            print(e)

    return data

In [6]:
for key in measurement_dictionary:
    measurement_dictionary[key] = dataSplit(measurement_dictionary[key],'dateTime')

#location_dictionary = measurement_dictionary 
location_dictionary = {}

# Flagging missed days 

In [7]:
dateStart = datetime.strptime(str(requestData['dateStart']) ,'%Y%m%d%H%M').date()
dateEnd = datetime.strptime(str(requestData['dateEnd']) ,'%Y%m%d%H%M').date()

dateList = pd.date_range(dateStart,dateEnd,freq='d')

timeStampRange = []
for day in dateList:
    timeStampRange.append(int(day.replace(tzinfo=timezone.utc).timestamp()))

In [8]:
for key in requestData['sensors']:
    timestampKeys = list(measurement_dictionary[key])

    #if there are no missing days skip to the next sensor
    if timeStampRange == timestampKeys:
        continue

    #flag the missing days
    else:
        missingDays = list(set(timeStampRange) - set(timestampKeys))
        print("Missing days for sensor %s :" % str(key) )
        for day in missingDays:
            print(datetime.utcfromtimestamp(day).date())


Missing days for sensor 814 :
2021-09-20
Missing days for sensor 821 :
2021-09-20


In [9]:
measurement_dictionary['814'].keys()

dict_keys([1631836800, 1631923200, 1632009600])

# Separating each list 

In [10]:
def calculateTimestamps(df):
    #set the index to Timestamp and automatically drops the datetime index
    df['Timestamp'] = df.index.values.astype(np.int64) // 10 ** 9
    df.set_index('Timestamp',inplace=True)
    return df

In [11]:
def dataframeAverages(intervalString,df):
    #convert all column types to int
    df = df.astype(int)
    #resample by minute and get mean
    df= df.resample(intervalString).mean() 

    df.isnull().sum() # identify any null values
    df.dropna(inplace=True) #drop null values
    #df = df.astype(int)  #convert avergaes back to int
    return df

In [12]:
for key in measurement_dictionary:
    temp_locations = {}
    temp_measurements = {}
    for timestamp in measurement_dictionary[key]:
        df_temp = measurement_dictionary[key][timestamp]
        df_temp.set_index('dateTime',inplace=True)
        df_temp.index = pd.to_datetime(df_temp.index,dayfirst=True)

        #extracting the location data
        ldf = df_temp[['latitude','longitude']]
        df_temp.drop(['latitude','longitude'],axis=1, inplace=True) #drop the location columns 

        #calculate minute averages
        df_temp = dataframeAverages('1min',df_temp)
        df_temp = calculateTimestamps(df_temp)

        #reassign new location dataframe to dictionary
        temp_locations[timestamp] = ldf

        #reassign new measurement dataframe to dictionary
        temp_measurements[timestamp] = df_temp

    location_dictionary[key] = temp_locations
    measurement_dictionary[key] = temp_measurements

    temp_locations = {} # clear dictionary 
    temp_measurements = {} # clear dictionary

# Preparing data for postgres

In [13]:
def calculateSensorMovement(ldf):
    precision = 0.00025 #this precision constant is derived from the GPS min and max readings which have 0.0003+/- variance for a stationary sensor 

    full_location_storage = False

    #check if the sensor has moved greater than a speicfied precision. 
    if ((ldf['latitude'].min() + precision) > ldf['latitude'].max()) or ((ldf['latitude'].max() - precision) < ldf['latitude'].min()):
        full_location_storage = True
    elif ((ldf['longitude'].min() + precision) > ldf['longitude'].max()) or ((ldf['longitude'].max() - precision) < ldf['longitude'].min()):
        full_location_storage = True
    else:
        full_location_storage = False

    return full_location_storage

In [14]:
def calculateBoundingBox(ldf):
    for row in ldf.iterrows():
        geometry_string = ""

        tempArray = [] 
        min_x= row[1][2]    # lat
        max_x = row[1][3]   # lat
                
        min_y= row[1][0]    # long
        max_y = row[1][1]   # long

    return "POLYGON((%f %f, %f %f, %f %f, %f %f,%f %f))" % (min_x,min_y,   min_x,max_y,   max_x,max_y,   max_x,min_y,   min_x,min_y)

In [15]:
sensor_summaries = {}
sensor_data = {}

for key in requestData['sensors']:
    
    df = mdf =  pd.DataFrame
    geometry_string = ""
    timestamp_sensor_key = ""

    #we can look into either dictioanry as they both share the same time range.
    for timestampKey in location_dictionary[key]:
   
        # concatenating numbers into text: 
        timestamp_sensor_key = "%s_%s" % (timestampKey, key)

        #try get location dataframe
        try:
            ldf = location_dictionary[key][timestampKey]
            
            #check if there is any movment in this current day of data and generate a bounding box if there is 
            if calculateSensorMovement(ldf) == True:
                print("Storing all measurements for this moving sensor")
                geometry_string = calculateBoundingBox(ldf)
            
            #if no movement is recorded them we can just take the min and max measuremnt for the day and drop the other values
            else:               
                temp_ldf = ldf.resample('D').agg(['min','max']) #resample on day interval
                geometry_string = calculateBoundingBox(temp_ldf)
               
                #convert all column types to float to get an averaged location
                ldf = ldf.astype(float)
                ldf = ldf.resample('D').mean()
                ldf = calculateTimestamps(ldf)
               
        except Exception as e:
            print('The dataframe is empty therefore no bounding box will be applied :{}'.format(e))
        
        # try get measurement dataframe
        try:
            mdf = measurement_dictionary[key][timestampKey] 
        except Exception as e:
            print('The measurement dataframe is empty. check csv files :{}'.format(e))


        #summaryArray = [timestamp_start,sensor_id,bouding_box,measurement_count]
        summaryArray = [timestampKey,int(key),geometry_string,len(mdf.index.values)] #inserting row into temp array
        sensor_summaries[timestamp_sensor_key] = summaryArray    #assign new dataframe to coressponding key

        #dataArray = [id, mesaurement_json,location_json]
        dataArray = [mdf.to_json(orient="columns"),ldf.to_json(orient="columns")] 
        sensor_data[timestamp_sensor_key] = dataArray    #assign new dataframe to coressponding key    

# Exporting to PostgresSQL 


# Writing records from a Dictionary of arrays to a SQL database
loop over all the keys and execute insert query

In [16]:
#TODO check if sensor is stationary. if it is then don't upload the location table and use the last known stationary geometry string (from our database) in each day of data

In [17]:
import psycopg2

In [18]:
#Connecting to an existing database
con = psycopg2.connect(
    host="localhost",
    database="airQuality",
    user="Riyad", 
    password="123",
    # attempt to connect for 3 seconds then raise exception
    connect_timeout = 3)

In [19]:
# cursor = con.cursor()

# query = "UPDATE sensor_network.sensor_summaries SET b_box = 'POLYGON ((52.0 10.0, 51.3 3.61, 51.3 3.0, 52.0 10.0))' WHERE timestamp_start = 1630454400 AND sensor_id = 31 \n RETURNING sensor_data_id;"
# #query = "SELECT sensor_data_id FROM sensor_network.sensor_summaries WHERE timestamp_start = 1630454400 AND sensor_id = 31"
# cursor.execute(query)
# con.commit()

# result = cursor.fetchone()

# print('Returns', result)

# if result == None:
#     #INSERT
#     print('INSERT')
# else:
#     #UPDATE
#     print(result)

# cursor.close()
# con.close()


In [20]:
#Opening a cursor to execute database operations
cursor = con.cursor()
query = "SELECT * FROM sensor_network.sensors"
#change dataframe to csv and save file
sensorsdf = pd.read_sql_query(query, con, index_col='lookup_id')
sensorsdf = sensorsdf.convert_dtypes() #convert to correct types
cursor.close()

In [21]:
sensorsdf.head()

,sensor_serial_number,id,type_id,active,last_update
lookup_id,,,,,
16701,02:00:00:00:40:45,5,1,False,None
18683,02:00:00:00:48:03,6,1,False,None
18720,02:00:00:00:48:28,8,1,False,None
18749,02:00:00:00:48:45,9,1,False,None
18704,02:00:00:00:48:18,10,1,False,None


In [22]:
# def Insertdata(mdf,ldf,sensor_id):
#     try:
#         #Opening a cursor to execute database operations
#         cursor = con.cursor()

#         #inserting sensor data and return the id of new record
#         cursor.execute("INSERT INTO sensor_data.archive_measurements (measurements,locations) VALUES(%s, %s) \n RETURNING id", (str(ldf[0]),str(ldf[1])) )
#         con.commit() 
        
#         #set id of new record into local variable
#         sensor_data_id = cursor.fetchone()[0]

#         #inserting a new sensor summary
#         cursor.execute("INSERT INTO sensor_network.sensor_summaries (timestamp_start, sensor_id, b_box, sensor_data_id, measurement_count) VALUES(%s, %s, %s, %s, %s)", (int(mdf[0]), int(sensor_id),str(mdf[2]),int(sensor_data_id),int(mdf[3])))
#         con.commit() 

#         #inserting sensor summary
#         current_date = datetime.fromtimestamp(int(key.split('_')[0]))
#         cursor.execute("UPDATE sensor_network.sensors SET last_update = %s WHERE id = %s", (current_date,int(sensor_id)))
#         con.commit() 

#         cursor.close()
#     #if table name does not exist exit loop 
#     except(psycopg2.errors.UndefinedTable) as error:
#         print('ERROR: ' + error)


In [23]:
def Insertdata(mdf,ldf,sensor_id,update):
    try:
        #Opening a cursor to execute database operations
        cursor = con.cursor()

        #inserting sensor data and return the id of new record
        cursor.execute("INSERT INTO sensor_data.archive_measurements (measurements,locations) VALUES(%s, %s) \n RETURNING id", (str(ldf[0]),str(ldf[1])) )
        con.commit() 
        
        #set id of new record into local variable
        sensor_data_id = cursor.fetchone()[0]

        if update == False:
            #inserting a new sensor summary
            cursor.execute("INSERT INTO sensor_network.sensor_summaries (timestamp_start, sensor_id, b_box, sensor_data_id, measurement_count) VALUES(%s, %s, %s, %s, %s)", (int(mdf[0]), int(sensor_id),str(mdf[2]),int(sensor_data_id),int(mdf[3])))
        else:
            #updating sensor summary
            cursor.execute("UPDATE sensor_network.sensor_summaries SET sensor_data_id = {} WHERE sensor_summaries.timestamp_start = {} AND sensor_summaries.sensor_id = {}".format(int(sensor_data_id),int(mdf[0]), int(sensor_id)))
        con.commit() 

        #inserting sensor summary
        current_date = datetime.fromtimestamp(int(key.split('_')[0]))
        cursor.execute("UPDATE sensor_network.sensors SET last_update = %s WHERE id = %s", (current_date,int(sensor_id)))
        con.commit() 

        cursor.close()
    #if table name does not exist exit loop 
    except(psycopg2.errors.UndefinedTable) as error:
        print('ERROR: ' + error)

In [24]:
previousSensorid = "x"
upsert = True

for key in sensor_summaries:

    #split key to get only sensorid
    s = key.split('_')[1]

    # print(s)
    # print(previousSensorid)
    if s != previousSensorid:
        previousSensorid = s
        upsert = True
    else:
        upsert = False

    #get the new key from sensors table
    sensor_id = sensorsdf.loc[int(s)]['id']

    mdf = sensor_summaries[key]
    ldf = sensor_data[key]

    if upsert == True:
        try:
            #Opening a cursor to execute database operations
            cursor = con.cursor()
            q = "INSERT INTO sensor_network.sensor_summaries (timestamp_start, sensor_id, b_box, sensor_data_id, measurement_count) VALUES({}, {}, {}, NULL, {})".format(int(mdf[0]), int(sensor_id),str("\'" + mdf[2] + "\'"),int(mdf[3]))
            q2 = "\n ON CONFLICT ON CONSTRAINT sensor_summaries_pkey"
            q3= " \n DO UPDATE SET b_box = {} WHERE sensor_summaries.timestamp_start = {} AND sensor_summaries.sensor_id = {} \n RETURNING sensor_data_id;".format(str("\'" + mdf[2] + "\'"),int(mdf[0]), int(sensor_id))
            query = q + q2 + q3
            
            cursor.execute(query)
            con.commit() 
            result = cursor.fetchone()
            #if the result fecthed is not empty then the sensor summary exsists and we can jsut update the raw measurement data
            if result[0] != None:           
                sensor_data_id = result[0]
                #update sensor data
                cursor.execute("UPDATE sensor_data.archive_measurements SET measurements = %s, locations = %s WHERE archive_measurements.id = %s;", (str(ldf[0]),str(ldf[1]),int(sensor_data_id)))
                con.commit()
                print('UPDATE at', result[0])

            else:
                Insertdata(mdf,ldf,sensor_id,True)
                print('INSERT')
        
        #if any errors occur then exit loop 
        except(Exception) as error:
            print('ERROR: ' + error)
            break

    #regular insert
    else: 
        Insertdata(mdf,ldf,sensor_id,False)
        print('INSERT')

#closing the connection
con.close()

UPDATE 781
INSERT
INSERT
UPDATE 784
INSERT
INSERT


In [25]:
# for key in sensor_summaries:  


#     #split key to get only sensorid
#     s = key.split('_')[1]

#     #get the new key from sensors table
#     sensor_id = sensorsdf.loc[int(s)]['id']

#     mdf = sensor_summaries[key]
#     ldf = sensor_data[key]
#     try:
#             #Opening a cursor to execute database operations
#             cursor = con.cursor()

#             #inserting sensor data and return the id of new record
#             cursor.execute("INSERT INTO sensor_data.archive_measurements (measurements,locations) VALUES(%s, %s) \n RETURNING id", (str(ldf[0]),str(ldf[1])) )
#             con.commit() 
            
#             #set id of new record into local variable
#             sensor_data_id = cursor.fetchone()[0]

#             #inserting sensor summary
#             cursor.execute("INSERT INTO sensor_network.sensor_summaries (timestamp_start, sensor_id, b_box, sensor_data_id, measurement_count) VALUES(%s, %s, %s, %s, %s)", (int(mdf[0]), int(sensor_id),str(mdf[2]),int(sensor_data_id),int(mdf[3])))
#             con.commit() 

#             #inserting sensor summary
#             current_date = datetime.fromtimestamp(int(key.split('_')[0]))
#             cursor.execute("UPDATE sensor_network.sensors SET last_update = %s WHERE id = %s", (current_date,int(sensor_id)))
#             con.commit() 

#             cursor.close()
#         #if table name does not exist exit loop 
#     except(psycopg2.errors.UndefinedTable) as error:
#         print('ERROR: ' + error)
#         break

# #closing the connection
# con.close()